In [1]:
import sys
sys.path.append('./codes/')
from config import args

from utils import *
from models import GCN
from metrics import *
args.dataset = 'syn3'

In [2]:
with open('./dataset/' + args.dataset + '.pkl', 'rb') as fin:
    adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, edge_label_matrix  = pkl.load(fin)

# Some preprocessing
if args.normfea:
    features = preprocess_features(features)
support = preprocess_adj(adj,args.normadj)

model = GCN(input_dim=features.shape[1], output_dim=y_train.shape[1])

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

features_tensor = tf.convert_to_tensor(features,dtype=tf.float32)
support_tensor = tf.SparseTensor(*support)
support_tensor = tf.cast(support_tensor,tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=tf.float32)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=tf.float32)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=tf.float32)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000
clip_value_min = -2.0
clip_value_max = 2.0


In [3]:
curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor,support_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss + args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
        cliped_grads = [tf.clip_by_value(t, clip_value_min, clip_value_max) for t in grads]
    optimizer.apply_gradients(zip(cliped_grads, model.trainable_variables))

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        if args.save_model:
            model.save_weights(args.save_path+args.dataset)
    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss), "train_acc=",
          "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc), "test_acc=", "{:.5f}".format(test_acc),
          "best_test_acc=", "{:.5f}".format(best_test_acc))

if not args.valid:
    model.save_weights(args.save_path + args.dataset)

Epoch: 0001 train_loss= 0.75557 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0002 train_loss= 0.74439 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0003 train_loss= 0.73437 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0004 train_loss= 0.72500 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0005 train_loss= 0.71658 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0006 train_loss= 0.70906 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0007 train_loss= 0.70246 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0008 train_loss= 0.69676 train_acc= 0.41810 val_acc= 0.39080 test_acc= 0.39773 best_test_acc= 0.39773
Epoch: 0009 train_loss= 0.69193 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0010 train_l

Epoch: 0089 train_loss= 0.58811 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0090 train_loss= 0.58735 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0091 train_loss= 0.58535 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0092 train_loss= 0.58433 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0093 train_loss= 0.58369 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0094 train_loss= 0.58152 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0095 train_loss= 0.58056 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0096 train_loss= 0.57946 train_acc= 0.66092 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0097 train_loss= 0.57820 train_acc= 0.66236 val_acc= 0.64368 test_acc= 0.62500 best_test_acc= 0.62500
Epoch: 0098 train_l

Epoch: 0177 train_loss= 0.46229 train_acc= 0.89943 val_acc= 0.90805 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0178 train_loss= 0.46225 train_acc= 0.89799 val_acc= 0.90805 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0179 train_loss= 0.46086 train_acc= 0.89655 val_acc= 0.89655 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0180 train_loss= 0.46036 train_acc= 0.89943 val_acc= 0.90805 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0181 train_loss= 0.46165 train_acc= 0.90661 val_acc= 0.90805 test_acc= 0.92045 best_test_acc= 0.92045
Epoch: 0182 train_loss= 0.45645 train_acc= 0.89799 val_acc= 0.90805 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0183 train_loss= 0.45957 train_acc= 0.89655 val_acc= 0.89655 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0184 train_loss= 0.45676 train_acc= 0.89799 val_acc= 0.90805 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0185 train_loss= 0.44985 train_acc= 0.89799 val_acc= 0.90805 test_acc= 0.90909 best_test_acc= 0.92045
Epoch: 0186 train_l

Epoch: 0266 train_loss= 0.36601 train_acc= 0.91379 val_acc= 0.93103 test_acc= 0.94318 best_test_acc= 0.94318
Epoch: 0267 train_loss= 0.36354 train_acc= 0.91379 val_acc= 0.93103 test_acc= 0.93182 best_test_acc= 0.94318
Epoch: 0268 train_loss= 0.36275 train_acc= 0.91379 val_acc= 0.93103 test_acc= 0.93182 best_test_acc= 0.94318
Epoch: 0269 train_loss= 0.36228 train_acc= 0.91379 val_acc= 0.93103 test_acc= 0.94318 best_test_acc= 0.94318
Epoch: 0270 train_loss= 0.36086 train_acc= 0.91236 val_acc= 0.93103 test_acc= 0.94318 best_test_acc= 0.94318
Epoch: 0271 train_loss= 0.36030 train_acc= 0.91810 val_acc= 0.93103 test_acc= 0.94318 best_test_acc= 0.94318
Epoch: 0272 train_loss= 0.35890 train_acc= 0.91379 val_acc= 0.93103 test_acc= 0.94318 best_test_acc= 0.94318
Epoch: 0273 train_loss= 0.35757 train_acc= 0.91379 val_acc= 0.93103 test_acc= 0.94318 best_test_acc= 0.94318
Epoch: 0274 train_loss= 0.35580 train_acc= 0.91523 val_acc= 0.93103 test_acc= 0.94318 best_test_acc= 0.94318
Epoch: 0275 train_l

Epoch: 0351 train_loss= 0.27374 train_acc= 0.97126 val_acc= 0.97701 test_acc= 0.98864 best_test_acc= 0.98864
Epoch: 0352 train_loss= 0.27359 train_acc= 0.97701 val_acc= 1.00000 test_acc= 0.98864 best_test_acc= 0.98864
Epoch: 0353 train_loss= 0.27196 train_acc= 0.96839 val_acc= 0.97701 test_acc= 0.97727 best_test_acc= 0.98864
Epoch: 0354 train_loss= 0.27054 train_acc= 0.96695 val_acc= 0.97701 test_acc= 0.97727 best_test_acc= 0.98864
Epoch: 0355 train_loss= 0.26884 train_acc= 0.97270 val_acc= 0.97701 test_acc= 0.98864 best_test_acc= 0.98864
Epoch: 0356 train_loss= 0.27034 train_acc= 0.97270 val_acc= 0.97701 test_acc= 0.98864 best_test_acc= 0.98864
Epoch: 0357 train_loss= 0.26565 train_acc= 0.96839 val_acc= 0.97701 test_acc= 0.97727 best_test_acc= 0.98864
Epoch: 0358 train_loss= 0.26694 train_acc= 0.97126 val_acc= 0.97701 test_acc= 0.98864 best_test_acc= 0.98864
Epoch: 0359 train_loss= 0.26565 train_acc= 0.97701 val_acc= 1.00000 test_acc= 0.98864 best_test_acc= 0.98864
Epoch: 0360 train_l

...

Epoch: 0603 train_loss= 0.09309 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0604 train_loss= 0.09331 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0605 train_loss= 0.09407 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0606 train_loss= 0.09530 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0607 train_loss= 0.09345 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0608 train_loss= 0.09402 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0609 train_loss= 0.09228 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0610 train_loss= 0.09241 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0611 train_loss= 0.09252 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0612 train_l

Epoch: 0691 train_loss= 0.08581 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0692 train_loss= 0.09625 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0693 train_loss= 0.09306 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0694 train_loss= 0.08300 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0695 train_loss= 0.09608 train_acc= 0.98132 val_acc= 0.98851 test_acc= 0.97727 best_test_acc= 0.98864
Epoch: 0696 train_loss= 0.08564 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0697 train_loss= 0.09107 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0698 train_loss= 0.08618 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0699 train_loss= 0.08583 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0700 train_l

Epoch: 0780 train_loss= 0.07216 train_acc= 0.98851 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0781 train_loss= 0.07099 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0782 train_loss= 0.07122 train_acc= 0.98132 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0783 train_loss= 0.07157 train_acc= 0.98851 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0784 train_loss= 0.06993 train_acc= 0.98132 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0785 train_loss= 0.07052 train_acc= 0.98132 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0786 train_loss= 0.07072 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0787 train_loss= 0.07178 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0788 train_loss= 0.06875 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0789 train_l

Epoch: 0869 train_loss= 0.06839 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0870 train_loss= 0.06590 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0871 train_loss= 0.06798 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0872 train_loss= 0.07143 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0873 train_loss= 0.06376 train_acc= 0.98132 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0874 train_loss= 0.06503 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0875 train_loss= 0.06321 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0876 train_loss= 0.06412 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0877 train_loss= 0.06388 train_acc= 0.98132 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0878 train_l

Epoch: 0955 train_loss= 0.06271 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0956 train_loss= 0.06353 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0957 train_loss= 0.05577 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0958 train_loss= 0.06603 train_acc= 0.99569 val_acc= 0.98851 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0959 train_loss= 0.05998 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0960 train_loss= 0.06086 train_acc= 0.97989 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0961 train_loss= 0.05620 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0962 train_loss= 0.05839 train_acc= 0.98994 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0963 train_loss= 0.05894 train_acc= 0.98132 val_acc= 1.00000 test_acc= 1.00000 best_test_acc= 0.98864
Epoch: 0964 train_l